## 1 - Imports

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.chdir("..")

In [ ]:
# %pip install tensorflow --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Cellule 1 — Imports
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, clear_output
from pathlib import Path

from src.utils.best_model_utils import load_best_table, load_best_model
from src.models.dl_utils import create_sequences, scale_data
from src.models.evaluation import evaluate_regression


## 2 - Configuration

In [5]:
import json
from urllib.request import urlopen

# Charger GeoJSON complet (17 CCAA)
geojson_url = "https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/spain-communities.geojson"
with urlopen(geojson_url) as resp:
    geojson_all = json.load(resp)

# Vérification rapide
print(geojson_all["type"])           # doit afficher "FeatureCollection"
print(len(geojson_all["features"]))  # doit afficher 17 (ou 19 si Ceuta/Melilla sont séparées)


FeatureCollection
19


In [7]:
import yaml

def load_config(config_path: str = "config.yaml") -> dict:
    """
    Lit le fichier YAML de configuration et renvoie un dict.
    """
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg

cfg = load_config("config.yaml")
proc_dir    = Path(cfg["paths"]["data"]["processed"])
reports_dir = Path("outputs/reports")

# Charger la table des meilleurs modèles
best_df = load_best_table()
zones   = best_df['zone'].tolist()
horizons = ["daily", "hourly"]


In [8]:
# Mapping GeoJSON name → clé zone
name_to_zone = {
    "Península Ibérica":      "nacional",
    "Illes Balears":          "Baleares",
    "Islas Canarias":         "Canarias",
    "Ceuta":                  "Ceuta",
    "Melilla":                "Melilla"
}

# Filtrer les features pour n’avoir que ces 5 zones
features_sel = [
    feat for feat in geojson_all["features"]
    if feat["properties"]["name"] in name_to_zone
]
geojson_sel = {"type":"FeatureCollection", "features":features_sel}

# Charger la table des meilleurs modèles et la liste complète des zones
best_df = load_best_table()
all_zones = best_df["zone"].tolist()  # ex. ['nacional','Baleares',…]

## 3 - Widgets de sélection

In [14]:
# Cellule — Widget de sélection + bouton
zone_widget    = widgets.Dropdown(
    options=all_zones,
    value="nacional",      # sélection par défaut
    description="Zone"
)
horizon_widget = widgets.Dropdown(
    options=["hourly","daily"],
    value="hourly",
    description="Horizon"
)
btn = widgets.Button(description="Voir les prévisions", button_style="success")

ui = widgets.HBox([zone_widget, horizon_widget, btn])
display(ui)

out = widgets.Output()
display(out)


Output()

In [15]:
# Cellule — Carte simplifiée
with out:
    fig = px.choropleth_mapbox(
        pd.DataFrame({
            "region": [feat["properties"]["name"] for feat in features_sel],
            "zone":   [name_to_zone[feat["properties"]["name"]] for feat in features_sel]
        }),
        geojson=geojson_sel,
        locations="region",
        featureidkey="properties.name",
        color="zone",
        mapbox_style="carto-positron",
        center={"lat":40,"lon":-3},
        zoom=4,
        opacity=0.5,
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()


In [19]:
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from src.utils.best_model_utils import load_best_model
from src.models.evaluation import evaluate_regression
from src.models.dl_utils import create_sequences, scale_data

def on_button_clicked(b):
    with out:
        clear_output(wait=True)
        
        # 1. Sélection zone/horizon
        zone    = zone_widget.value
        horizon = horizon_widget.value
        
        # 2. Charger les données proces­sées
        proc_dir = Path("data/submission")
        df = pd.read_parquet(proc_dir / f"{zone}_processed_{horizon}.parquet")
        
        # 3. Séparer train/test 80/20
        split_idx = int(len(df) * 0.8)
        df_test   = df.iloc[split_idx:].copy()
        
        # 4. Charger le modèle gagnant
        model = load_best_model(zone, horizon)
        
        # 5. Préparer X_test et y_true
        y_true = df_test["demand"].values
        dates  = df_test.index
        
        # 6. Prédictions selon le type de modèle
        if hasattr(model, "predict") and model.__class__.__module__.startswith("tensorflow"):
            # → Deep Learning : besoin de séquences + scaling
            LOOKBACK     = 24 if horizon == "hourly" else 7
            feature_cols = [c for c in df.columns if c != "demand"]
            
            # On recrée les séquences sur l'ensemble et on aligne la fin
            X_all, y_all = create_sequences(df, "demand", feature_cols, LOOKBACK, horizon == "daily" and 7 or 1)
            n_test        = len(df_test)
            X_test_seq    = X_all[-n_test:]
            
            # Scale
            _, X_test_scaled, _ = scale_data(X_all[:-n_test], X_test_seq)
            y_pred = model.predict(X_test_scaled).ravel()
        
        elif getattr(model, "__module__", "").startswith("pmdarima"):
            # → ARIMA / SARIMAX
            # pmdarima attend n_periods
            y_pred = model.predict(n_periods=len(df_test))
        
        else:
            # → scikit‑learn : on passe X_test
            X_test = df_test.drop(columns="demand")
            y_pred = model.predict(X_test)
        
        # 7. Tracer Réel vs Prédit
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=dates, y=y_true, name="Réel"))
        fig.add_trace(go.Scatter(x=dates, y=y_pred, name="Prédit"))
        fig.update_layout(
            title=f"Prévisions {horizon} – {zone}",
            xaxis_title="Date",
            yaxis_title="Demande"
        )
        fig.show()
        
        # 8. KPI
        kpi = evaluate_regression(
            model=None, X_test=None,
            y_test=pd.Series(y_true),
            pred_series=pd.Series(y_pred)
        )
        print(f"MAE:  {kpi['MAE']:.2f}")
        print(f"RMSE: {kpi['RMSE']:.2f}")
        # print(f"MAPE: {kpi['MAPE']:.1f}%")
        
# Lier le bouton
btn.on_click(on_button_clicked)


## 4 - Fonction de mise à jour

In [21]:
out = widgets.Output()

def update_dashboard(change):
    out.clear_output()
    zone    = zone_widget.value
    horizon = horizon_widget.value

    # 1. Chargement des données
    df = pd.read_parquet(proc_dir / f"{zone}_processed_{horizon}.parquet")
    split = int(len(df)*0.8)
    df_train, df_test = df.iloc[:split], df.iloc[split:]

    # 2. Affichage Stats
    with out:
        print(f"### Statistiques pour {zone} ({horizon})")
        display(df["demand"].describe())

        # 3. Charger et afficher le meilleur modèle
        model = load_best_model(zone, horizon)
        print(f"**Modèle sélectionné** : {type(model).__name__}")

        # 4. Préparation des données de test
        if hasattr(model, "predict") and model.__class__.__module__.startswith("tensorflow"):
            # Deep Learning
            LOOKBACK = 24 if horizon=="hourly" else 7
            features = [c for c in df.columns if c!="demand"]
            X_seq, y_seq = create_sequences(df, "demand", features, LOOKBACK)
            n_test = len(df_test)
            X_test_seq = X_seq[-n_test:]
            _, X_test_seq_scaled, _ = scale_data(X_seq[:-n_test], X_test_seq)
            y_true = y_seq[-n_test:].ravel()
            y_pred = model.predict(X_test_seq_scaled).ravel()
            idx = df_test.index
        else:
            # Sklearn / ARIMA
            X_test = df_test.drop(columns="demand")
            try:
                y_pred = model.predict(X_test)
            except:
                y_pred = model.predict(n_periods=len(df_test))
            y_true = df_test["demand"].values
            idx    = df_test.index

        # 5. Plot Réel vs Prédiction
        import matplotlib.pyplot as plt
        # plt.style.use("seaborn-v0_8")
        fig, ax = plt.subplots(figsize=(10,4))
        ax.plot(idx, y_true, label="Réel")
        ax.plot(idx, y_pred, label="Prédit", alpha=0.7)
        ax.set_title(f"Demande — {zone} ({horizon})")
        ax.legend()
        plt.show()

        # 6. KPI
        kpi = evaluate_regression(None, None, pd.Series(y_true), pd.Series(y_pred))
        print("**Performance**")
        print(f"MAE:  {kpi['MAE']:.2f}")
        print(f"RMSE: {kpi['RMSE']:.2f}")
        print(f"MAPE: {kpi['MAPE']:.2f}%")

widgets.interact(update_dashboard, change=zone_widget)  # lien zone
widgets.interact(update_dashboard, change=horizon_widget)  # lien horizon
display(out)


interactive(children=(Dropdown(description='Zone', index=11, options=('Peninsule_Iberique', 'Baleares', 'Canar…

interactive(children=(Dropdown(description='Horizon', options=('hourly', 'daily'), value='hourly'), Output()),…

Output()

## 6 - Lancer l’application Streamlit

In [3]:
import os
print("Démarrage du dashboard Streamlit… (Ctrl+C pour arrêter)")
os.system("streamlit run dashboard/app.py")


Démarrage du dashboard Streamlit… (Ctrl+C pour arrêter)


2